In [2]:

AX1="A → (B → A)"
AX2="(A → (B → C)) → ((A → B) → (A → C))"
AX3="(¬B → ¬A) → (A → B)"


from sympy.logic.boolalg import Or, And, Not, Implies
from sympy import symbols, sympify
from sympy.logic import simplify_logic
from sympy.parsing.sympy_parser import parse_expr
import re

class RawImplies(Implies):
    @classmethod
    def eval(cls, lhs, rhs):
        # completely disable evaluation
        return None

def remove_outer_brackets(s):
    if not s or s[0] != '(' or s[-1] != ')':
        return s  

    depth = 0
    for i, char in enumerate(s):
        if char == '(':
            depth += 1
        elif char == ')':
            depth -= 1
        if depth == 0 and i < len(s) - 1: # not fully wrapped
            return s
    return s[1:-1] # fully wrapped

def parser(s):
    if type(s) == list:
        s = "".join(f"({item})" for item in s)
    if type(s)!=str:
        print("Parsing error, wrong type entered")
    s=remove_outer_brackets(s)
    s=s.replace(" ", "")
    stack = []
    result = []
    skip=False
    current_str=""
    for i, char in enumerate(s):
        if skip:
            skip=False
            continue
        if char == '(':
            stack.append(i)
        elif char == ')':
            if stack:
                start = stack.pop()
                # Extract the content inside the current matching parentheses
                if stack==[]:
                    result.append(s[start + 1:i])
        elif stack==[]:
            if char == "¬" and (len(s)>2) and i!=len(s)-1 and s[i+1].isalpha():
                # print(s[i:i+2],"eeee")
                result.append(s[i:i+2])
                skip=True
            else:
                result.append(char)
    # print(s,result)
    return result
    
def make_sympy(s:str):
    array=parser(s)
    # print(array)
    answer=None
    if len(array)==1 and isinstance(array[0], str) and len(array[0]) == 1:
        char=array[0]
        if not char.isalpha():
            print("Error: lone floating operator",char)
            return
        return symbols(char)
    for i,item in enumerate(array):
        if item == "¬":
            if i==len(array)-1:
                print("Error: floating ¬")
                return
            # print(item,array[i+1],"!!!!!!!!!")
            answer=Not(make_sympy(array[i+1]), evaluate=False)
            # print(item,array[i+1],":")
            # print(answer,"..........")
            # wont nessisarily break
        if item == "→":
            if i==0 or i==len(array)-1:
                print("Error floating →")
                return
            if answer:
                # print(answer,"→",make_sympy(array[i+1:]),":")
                answer=RawImplies(answer,make_sympy(array[i+1:]), evaluate=False)
                # print(answer,"......")
            else:
                # print(make_sympy(array[i-1]),"→",make_sympy(array[i+1:]),":")
                answer=RawImplies(make_sympy(array[i-1]),make_sympy(array[i+1:]), evaluate=False)
                # print(answer,".....")
            break
        # if i==len(array)-1:
            # print("ERROR no operators")
    return answer

# print(make_sympy("¬(¬B → ¬A) → ¬(A → B)"))

axiom1=make_sympy(AX1)
axiom2=make_sympy(AX2)
axiom3=make_sympy(AX3)

In [14]:
class proof_line:
    def __init__(self,expr,step):
        self.expr=expr
        self.step=step

def check_proof(lines):
    works=True
    for i,line in enumerate(lines):
        if i==0:
            continue
        step=line.step.replace(" ","")
        # print("Testing:",step,line.expr)
        if step.lower()=="premise":
            continue
        elif step.lower()=="ax1":
            if line.expr != axiom1:
                print("incorrect use of AX1: ",line.expr)
                works=False
                break
        elif step.lower()=="ax2":
            if line.expr != axiom2:
                print("incorrect use of AX2: ",line.expr)
                works=False
                break
        elif step.lower()=="ax3":
            if line.expr != axiom3:
                print("incorrect use of AX3: ",line.expr)
                works=False
                break
        elif re.search(r"sub[0-9]+\[([a-z]:=[a-z¬→\(\),]+)+\]",step.lower()):
            substr=re.search(r"\[([a-zA-Z]:=[a-zA-Z¬→\(\),]+)+\]",step).group(0).replace("[", "").replace("]", "")
            subint=int(re.search(r"[0-9]+",step).group())
            if subint>=i or subint<=0:
                print(f"Incorrect subsitution reference: {line.step} in line {i}.")
                works=False
                break
            items = [item.strip() for item in substr.split(',')]
            # print(items)
            subby=lines[subint].expr
            for item in items:
                thing=re.search(r"([a-zA-Z]):=([a-zA-Z¬→\(\)]+)",item)
                LHS=symbols(thing.group(1))
                RHS=make_sympy(thing.group(2))
                subby=subby.subs(LHS,RHS)
            if subby!=line.expr:
                print(f"Incorrect subsitution:\n{line.step} on {lines[subint].expr} does not give {line.expr}")
                works=False
                print("NO SUBSITUION WRONG!!!")
                print(subby)
                print(line.expr)
                break
        elif re.search(r"mp[0-9]+,[0-9]+",step.lower()):
            mpstr=re.search(r"mp([0-9]+),([0-9]+)",step.lower())
            mpint1=int(mpstr.group(1))
            mpint2=int(mpstr.group(2))
            if mpint1>=i or mpint2>=i or mpint1<=0 or mpint2<=0:
                print(f"Incorrect Modus ponens reference: {line.step} in line {i}.")
                works=False
                break
            mp1=lines[mpint1].expr
            mp2=lines[mpint2].expr
            if mp1.func!=RawImplies and mp2.func!=RawImplies:
                works=False
                break
            if mp1.func==RawImplies:
                LHS,RHS=mp1.args
                if LHS==mp2 and RHS==line.expr:
                    works = True
                    break
                else:
                    works=False
            if mp2.func==RawImplies:
                LHS,RHS=mp2.args
                if LHS==mp1 and RHS==line.expr:
                    works = True
                    break
                else:
                    works=False
            if not(works):
                print(f"Incorrect Modes Ponens use:\n{mp1} and {mp2} does not give {line.expr}")
        else: 
            print("AAAAAAA",step.lower())
            works=False
            break
    if works:
        if lines[0].expr == lines[-1].expr:
            print("yippeee valid proof!")
        else:
            print("end statement does not match")
    else:
        print("WOMP WOMP")


In [4]:
test_proof=[
proof_line(make_sympy("((¬p → q) → (¬p → ¬p))"),""),
proof_line(make_sympy("(A → (B → C)) → ((A → B) → (A → C))"),"AX2"),
proof_line(make_sympy("(¬p → (B → C)) → ((¬p → B) → (¬p → C))"),"Sub1[A:=¬p]"),
proof_line(make_sympy("(¬p → (q → ¬p)) → ((¬p → q) → (¬p → ¬p))"),"Sub2[C:=¬p,B:=q]"),
proof_line(make_sympy("(A → (B → A))"),"ax 1"),
proof_line(make_sympy("(¬p → (q → ¬p))"),"Sub4[A:=¬p,B:=q]"),
proof_line(make_sympy("((¬p → q) → (¬p → ¬p))"),"MP3,5")]

check_proof(test_proof)

yippeee valid proof


In [5]:
user_input = "((¬p → q) → (¬p → ¬p))"

In [1]:
!pip install ollama

In [1]:
import ollama

client=ollama.Client()

model="Prop_llama"

In [ ]:
response=client.generate(model=model,prompt=user_input)

print(response.response)

1. (A → (B → C)) → ((A → B) → (A → C))...AX2
2. (¬p → (q → ¬p)) → ((¬p → q) → (¬p → ¬p))...Sub1[A:=¬p,C:=¬p,B:=q]
3. (A → (B → A))...AX1
4. (¬p → (q → ¬p))...Sub3[A:=¬p,B:=q]
5. ((¬p → q) → (¬p → ¬p))...MP2,4


In [16]:
def make_and_check_proof():
    target_str=input("Paste your desired formula using →,¬ symbols: ")
    print(target_str," to be proven")
    target=make_sympy(target_str)
    proof=client.generate(model=model,prompt=target_str)
    print("Generated response:\n",proof.response)
    proof=(proof.response).splitlines()
    proof_array=[]
    proof_array.append(proof_line(target,"to be proven"))
    for i,line in enumerate(proof):
        line=line.replace(" ","")
        num=re.search(r"([0-9]+).",line)
        if not(num):
            continue
        stuff=re.search(r"([\(\)A-Za-z→¬]+)...([A-Za-z0-9,\[\]:=→¬]+)",line)
        expr=stuff.group(1)
        # print("extracted expr: ",expr)
        expr=make_sympy(expr)
        step=stuff.group(2)
        # print("Step: ",step)
        proof_array.append(proof_line(expr,step))
    check_proof(proof_array)
    
make_and_check_proof()

((¬p → q) → (¬p → ¬p))  to be proven
Generated response:
 1. (A → (B → C)) → ((A → B) → (A → C))...AX2
2. (¬p → q) → ((¬p → p) → (¬p → ¬p))...Sub2[A:=¬p,B:=q,C:=¬p]
3. (A → (B → A))...AX1
4. (¬p → (p → ¬p))...Sub3[A:=¬p,B:=p]
5. ((¬p → p) → (¬p → ¬p))...MP2,4
6. (¬p → q) → ((¬p → p) → (¬p → ¬p))...Premise Introduction
7. ((¬p → q) → (¬p → ¬p))...Modus Ponens 1,6
Incorrect subsitution reference: Sub2[A:=¬p,B:=q,C:=¬p] in line 2.
WOMP WOMP
